In [1]:
import numpy as np
import random

### Генератор Лемера

In [ ]:
class Linear_Low:
    a = 2**16 + 1
    c = 119
    x0 = 1  # тюряга #

    def generate_bytes(self, n: int):
        seq = np.zeros(n, dtype=np.uint32)
        seq[0] = self.x0

        for i in range(0, n - 1):
            seq[i+1] = (self.a*seq[i] + self.c) 

        seq = seq % (2**8)

        return seq

class Linear_High:
    a = 2**16 + 1
    c = 119
    x0 = 1  # тюряга #

    def generate_bytes(self, n: int):
        seq = np.zeros(n, dtype=np.uint32)
        seq[0] = self.x0

        for i in range(0, n - 1):
            seq[i+1] = (self.a*seq[i] + self.c)

        seq = (seq >> 24)

        return seq
        
            

In [ ]:
de1 = Linear_Low()
de2 = Linear_High()

print(de1.generate_bytes(20))
print(de2.generate_bytes(20))

### Генератор L20

In [ ]:
class L20:
    def __init__(self, x_init: np.array):
        self.x_init = x_init

    def generate_bits(self, n: int):
        seq = np.concatenate([np.array(self.x_init, dtype=np.uint8), np.zeros(n - 20, dtype=np.uint8)])

        for i in range(20, n):
            seq[i] = seq[i - 3] ^ seq[i - 5] ^ seq[i - 9] ^ seq[i - 20]

        seq = seq % 2

        return seq
            

In [ ]:
smp = np.array([random.randint(0, 1) for _ in range(20)])
print(smp)
de_L20 = L20(smp)

print(de_L20.generate_bits(100))

### Генератор L89

In [ ]:
class L89:
    def __init__(self, x_init: np.array):
        self.x_init = x_init

    def generate_bits(self, n: int):
        seq = np.concatenate([np.array(self.x_init, dtype=np.uint8), np.zeros(n - 89, dtype=np.uint8)])

        for i in range(89, n):
            seq[i] = seq[i - 38] ^ seq[i - 89]

        seq = seq % 2

        return seq
            

In [ ]:
smp = np.array([random.randint(0, 1) for _ in range(89)])
de_L89 = L89(smp)

print(de_L89.generate_bits(90))

### Генератор Вольфрама

In [ ]:
# В ПІТОНІ НЕМА ВБУДОВАНОГО ЦИКЛІЧНОГО ЗСУВУ
def rcs(n: np.uint32, rotations):
    return (n >> rotations | n << (32-rotations))

def lcs(n: np.uint32, rotations):
    return (n << rotations | n >> (32-rotations))

class Wolfram:
    def __init__(self, r0: np.uint32):
        self.r0 = r0

    def generate_bits(self, n: int):
        r_i = self.r0
        seq = np.zeros(n, dtype=np.uint8)

        for i in range(0, n):
            seq[i] = r_i % 2
            r_i = lcs(r_i, 1) ^ (r_i | rcs(r_i, 1))  


        return seq
            

In [ ]:
de_wolfram = Wolfram(1)

print(de_wolfram.generate_bits(100))

### Генератор BM

In [ ]:
class BM:
    def __init__(self, p, a):
        self.p = p
        self.a = a

    def generate_bits(self, n: int):
        seq = np.zeros(n, dtype=object)
        seq[0] = random.randint(0, self.p - 1) 

        for i in range(1, n):
            seq[i] = pow(self.a, seq[i - 1], self.p)

        seq = np.array([seq < (self.p - 1) / 2], dtype=np.uint8) 

        return seq
    
    def generate_bytes(self, n: int):
        seq = np.zeros(n, dtype=object)
        seq[0] = random.randint(0, self.p - 1) 

        for i in range(1, n):
            seq[i] = pow(self.a, seq[i - 1], self.p)

        seq = np.array([seq < (self.p - 1) / 2], dtype=np.uint8) 

        return seq
            

In [6]:


print(np.array([1, 2, 3, 4], dtype=np.uint8) < 3)

[ True  True False False]


#### Криптографічно нестійкий генератор

#### Криптографічно стійкий генератор